# Testing a new version of `simulate` that uses numba compilation

In [1]:
import numpy as np, pandas as pd
from numba import jit
from pynamical import simulate as simulate_old, cubic_map as cubic_map_old

In [2]:
# decorate functions with numba jit
@jit(cache=True, nopython=True)
def logistic_map_jit(pop, rate):
    return pop * rate * (1 - pop)

In [3]:
@jit(cache=True, nopython=True)
def cubic_map_jit(pop, rate):
    return rate * pop ** 3 + pop * (1 - rate)

In [4]:
# you can't pass a jitted function to a jitted function unless you turn off 'nopython' mode (which makes it slow)
# in other words, you can't pass different model functions directly to the simulate function
# instead, use a closure: 
# the make_simulator function returns a jitted simulator function that receives the jitted model function
# without it being an argument passed to the simulator function, because of the closure local scope
def make_simulator(model, num_gens=50, rate_min=0.5, rate_max=4, num_rates=8, num_discard=0, initial_pop=0.5):
    
    @jit(cache=True, nopython=True)
    def simulator(num_gens=num_gens, rate_min=rate_min, rate_max=rate_max, num_rates=num_rates, 
                  num_discard=num_discard, initial_pop=initial_pop):
        
        pops = np.empty(shape=(num_gens*num_rates, 2), dtype=np.float64)
        rates = np.linspace(rate_min, rate_max, num_rates)

        # for each rate, run the function repeatedly, starting at the initial_pop
        for rate_num, rate in zip(range(len(rates)), rates):
            pop = initial_pop

            # first run it num_discard times and ignore the results
            for _ in range(num_discard):
                pop = model(pop, rate)

            # now that those gens are discarded, run it num_gens times and keep the results
            for gen_num in range(num_gens):
                row_num = gen_num + num_gens * rate_num
                pops[row_num] = [rate, pop]
                pop = model(pop, rate)
        
        return pops
    
    return simulator

In [5]:
def simulate_jit(model=logistic_map_jit, num_gens=50, rate_min=0.5, rate_max=4, num_rates=8, num_discard=0, initial_pop=0.5):
    
    # make the jitted simulator
    simulator = make_simulator(model=model, num_gens=num_gens, rate_min=rate_min, rate_max=rate_max, 
                               num_rates=num_rates, num_discard=num_discard, initial_pop=initial_pop)
    
    # run the simulator to create the pops to pass to the dataframe
    pops = simulator()
    
    # return a dataframe with one column for each growth rate and one row for each timestep (aka generation)
    df = pd.DataFrame(data=pops, columns=['rate', 'pop'])
    df.index = pd.MultiIndex.from_arrays([num_rates * list(range(num_gens)), df['rate'].values])
    return df.drop(labels='rate', axis=1).unstack()['pop']

## Now test the original version vs the Numba JIT version

In [6]:
num_gens = 1000
num_rates = 1000

In [7]:
%timeit simulate_old(num_gens=num_gens, num_rates=num_rates)

1 loop, best of 3: 1.43 s per loop


In [8]:
%timeit simulate_jit(num_gens=num_gens, num_rates=num_rates)

1 loop, best of 3: 878 ms per loop


With the logistic map, the JIT version takes about 60% of the time to execute compared to the original version.

Now try the cubic map:

In [9]:
%timeit simulate_old(model=cubic_map_old, num_gens=num_gens, num_rates=num_rates)

1 loop, best of 3: 1.72 s per loop


In [10]:
%timeit simulate_jit(model=cubic_map_jit, num_gens=num_gens, num_rates=num_rates)

1 loop, best of 3: 880 ms per loop


With the cubic map, the JIT version takes about 50% of the time to execute compared to the original version.